I want to integrate classical methods to optimize hyperparameters from quantum world.

In [1]:
# Install Qiskit Dynamics
!pip install qiskit-dynamics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.8/191.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 2.8 MB/s eta 0:00:00


In [16]:
import numpy as np
from qiskit_dynamics import Solver, Signal
from qiskit_dynamics.models import LindbladModel
from qiskit.quantum_info import Operator, Statevector, DensityMatrix, Pauli
import matplotlib.pyplot as plt


In [23]:
N = 6  # Number of qubits
omega_z = 2 * np.pi * 5e9  # Qubit frequency in rad/s (5 GHz)
omega_x = 2 * np.pi * 0.5e9  # Drive frequency in rad/s (0.5 GHz)
J = 2 * np.pi * 0.2e9  # Coupling strength in rad/s (0.2 GHz)
Gamma = 2 * np.pi * 1e6  # Dissipation rate in rad/s (1 MHz)
#initializing the operators X,Y, Z
x_ops = []
y_ops = []
z_ops = []
qubits = range(N)
zeros = Operator(np.zeros((2 ** N, 2 ** N)))

for i in qubits:
    X = zeros + Pauli('X')(i)
    x_ops.append(X)

    Y = zeros + Pauli('Y')(i)
    y_ops.append(Y)

    Z = zeros + Pauli('Z')(i)
    z_ops.append(Z)

In [24]:
H = zeros
for i in qubits:
    X = x_ops[i]
    Z = z_ops[i]
    H += 0.5 * omega_x * X  # Driving term
    H += 0.5 * omega_z * Z  # Qubit energy term

    if N > 1:
        j = i + 1 if i < (N - 1) else 0
        op_xx = zeros + Pauli('XX')(i, j)
        op_yy = zeros + Pauli('YY')(i, j)
        H += 0.5 * J * (op_xx + op_yy)  # Interaction terms

In [25]:
# THE Lindblad operators (decay process)
L_ops = []
for i in qubits:
    X = x_ops[i]
    Y = y_ops[i]
    L_ops.append(np.sqrt(Gamma) * 0.5 * (X + 1j * Y))

In [ ]:
solver = Solver(static_hamiltonian=H, static_dissipators=L_ops)

# all qubits in ground state
y0 = DensityMatrix.from_label('0' * N)

t_final = 2.0e-6  # Total simulation time (e.g., 2 microseconds)
tau = 0.01e-6  # Time step (10 ns)
n_steps = int(np.ceil(t_final / tau)) + 1
t_eval = np.linspace(0., t_final, n_steps)

# Solve the Lindblad equation
sol = solver.solve(t_span=[0., t_final], y0=y0, t_eval=t_eval)

In [ ]:
# Calculate expectation values
n_times = len(sol.y)
x_data = np.zeros((N, n_times))
y_data = np.zeros((N, n_times))
z_data = np.zeros((N, n_times))
x_mean = np.zeros((n_times,))
y_mean = np.zeros((n_times,))
z_mean = np.zeros((n_times,))

for t_i, sol_t in enumerate(sol.y):
    for qubit, obs in enumerate(x_ops):
        x_data[qubit, t_i] = sol_t.expectation_value(obs).real
    x_mean[t_i] = np.mean(x_data[:, t_i])

    for qubit, obs in enumerate(y_ops):
        y_data[qubit, t_i] = sol_t.expectation_value(obs).real
    y_mean[t_i] = np.mean(y_data[:, t_i])

    for qubit, obs in enumerate(z_ops):
        z_data[qubit, t_i] = sol_t.expectation_value(obs).real
    z_mean[t_i] = np.mean(z_data[:, t_i])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(t_eval, x_mean, label='$ N^{-1}\\sum_i \\langle X_i \\rangle$')
plt.plot(t_eval, y_mean, label='$ N^{-1}\\sum_i \\langle Y_i \\rangle$')
plt.plot(t_eval, z_mean, label='$ N^{-1}\\sum_i \\langle Z_i \\rangle$')
plt.legend(fontsize=16)
plt.xlabel('$t$ (seconds)', fontsize=16)
plt.ylabel('Expectation values', fontsize=16)
plt.title('Mean Bloch vector vs. $t$', fontsize=16)
plt.grid(True)
plt.show()